# Agent-Based Modeling Digitial Day of Learning Workshop
## Forrest Fire Model in Mesa
### Joseph Engler, PhD & Chris Reuter


For this exercise we will use a python library called Mesa. Mesa was created specifically for Agent-based modeling and is the standard at many research institutions globally.

In [2]:
!pip install --proxy http://proxy.rockwellcollins.com:9090 mesa

In [3]:
import mesa
import matplotlib.pyplot as plt
import nest_asyncio
import warnings

warnings.filterwarnings('ignore')
nest_asyncio.apply()

# Creating a the Agent-Based Model
The Agent-based model is performed on a grid. Each cell of the grid can have various properties and components of the model. Once the model is completely setup then we can run the model. Running the model consists of individual time steps in which the model, agents, and other components perform some tasks. We will begin with creating the agents of the model.

## Creating Agents
Mesa uses agents as the main object of modelling. We will create an agent that will represent a section of the forest. Agents are a class in Mesa that we will extend and pass it the required variables of the model and the agent's unique id. Agents also have a method called step that we need to overload. During the step the agent will determine if it is on fire and if it is, then it will light its neighbors on fire.

In [4]:
class Forest(mesa.Agent):
    def __init__(self, unique_id, model, position):
        super().__init__(unique_id, model)
        self.pos = position
        self.status = "Fine"
        
    def step(self):
        if self.status == "Burning":
            for neighbor in self.model.grid.iter_neighbors(self.pos, True):
                if neighbor.status == "Fine":
                    if self.model.random.random() > 0.75:
                        neighbor.status = "Burning"
                self.status = "Burned"
        
        #TODO
        #Develop logic for regenerating the forest after being burnt
    
    

## Creating the Model
The model is a Mesa class that we will extend just like we did with the agents. The model is the controller of the overall Agent-based Model

In [5]:
class ForestFire(mesa.Model):
    def __init__(
        self,
        width = 100,
        height = 100,
        density=0.65):
        
        super().__init__()
        self.width = width
        self.height = height
        self.density = density
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.SingleGrid(self.width, self.height, torus=False)
        self.datacollector = mesa.DataCollector(
            model_reporters={
                "Fine": lambda model: self.count_type(model, "Fine"),
                "Burning": lambda model: self.count_type(model, "Burning"),
                "Burned": lambda model: self.count_type(model, "Burned"),
                #TODO: Add output for regeneration
            }
        )
        self._initialize_forest()
        self.datacollector.collect(self)
        
        
    def _initialize_forest(self):
        for _contents, pos in self.grid.coord_iter():
            forest = Forest(self.next_id(), self, pos)
            if self.random.random() < self.density:
                if pos[0] == 0: #set this section to burning
                    forest.status = "Burning"
                else:
                    forest.status = "Fine"
            else:
                forest.status = "Burned"
            self.schedule.add(forest)
            self.grid.place_agent(forest, pos)
            
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        
    @staticmethod
    def count_type(model, status):
        count = 0
        for forest in model.schedule.agents:
            if forest.status == status:
                count += 1
        return count
                    
    

## Create the Server to Run the Model


In [ ]:
GRID_WIDTH = 100
GRID_HEIGHT = 100
CANVAS_WIDTH = 500
CANVAS_HEIGHT = 500

COLORS = {
    "Fine": "#00AA00",
    "Burning": "#FF0000",
    "Burned": "#3D2B1F",
}

def forest_portrayal(forest):
    if not forest:
        return

    (x, y) = forest.pos
    return {
        "Shape": "rect",
        "w": 1,
        "h": 1,
        "Filled": True,
        "Layer": 0,
        "x": x,
        "y": y,
        "Color": COLORS[forest.status],
    }

canvas_element = mesa.visualization.CanvasGrid(
    forest_portrayal, GRID_WIDTH, GRID_HEIGHT, CANVAS_WIDTH, CANVAS_HEIGHT
)

forest_chart = mesa.visualization.ChartModule(
    [{"Label": label, "Color": color} for (label, color) in COLORS.items()]
)

pie_chart = mesa.visualization.PieChartModule(
    [{"Label": label, "Color": color} for (label, color) in COLORS.items()]
)

model_params = {
    "width": GRID_WIDTH,
    "height": GRID_HEIGHT,
    "density": mesa.visualization.Slider("Forest Density", 0.65, 0.01, 1.0, 0.01),
    #TODO: Add a slider that will determine the speed at which the forests regenerate
}



server = mesa.visualization.ModularServer(
    ForestFire, [canvas_element, forest_chart, pie_chart], "Forest Fire", model_params, port=8522
)

In [ ]:
server.launch(open_browser=True)

# Power of Experimentation

The power of Agent-based Modeling is that you can easily explore many hypotheses and determine how they effective they are by adding them to the model and running it. 

Using the model and agent definitions above, you can explore many other scenarios by which forest fire damage can be mitigated. Try adding a status of "Fire Break" that can not catch fire and see how that benefits or hurts the forests during fire conditions. 

